# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/')

In [6]:
filepath = song_files[0]
filepath2 = song_files[1]
print(filepath)
print(filepath2)

/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json
/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json


In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[[df.columns[7], df.columns[8], df.columns[0], df.columns[9], df.columns[5]]]
song_data = song_data.drop_duplicates(keep='first')
song_data = song_data.values[0]
song_data

array(['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008,
       149.86404], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_longitude', 'artist_latitude']].values[0]
artist_data = list(artist_data)
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files('data/log_data/')

In [6]:
filepath = log_files[0]
filepath2 = log_files[1]
print(filepath)
print(filepath2)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json
/home/workspace/data/log_data/2018/11/2018-11-27-events.json


In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [8]:
df = df.query("page == 'NextSong'")
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [9]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [10]:
##print(t, t.dt.hour, t.dt.day, t.dt.week)
t_hour = t.dt.hour
t_day  = t.dt.day
t_week = t.dt.week
t_month = t.dt.month
t_year = t.dt.year
t_weekday = t.dt.weekday
time_data = (t, t_hour, t_day, t_week, t_month, t_year, t_weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year','weekday')

In [11]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [12]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [13]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates(keep='first')

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [14]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [15]:
for index, row in df.iterrows():
    #print(row)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        print('song ' + str(row.song), 'artist ' + str(row.artist), 'length ' + str(row.length))
        print(songid, artistid)
    # insert songplay record
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    #cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

song Ain't No Sunshine artist Sydney Youngblood length 238.07955
None None
song My Advice 2 You (Explicit) artist Gang Starr length 151.92771
None None
song My First Kiss (Feat. Ke$ha) [Album Version] artist 3OH!3 length 192.522
None None
song The Girl and The Robot artist RÃÂ¶yksopp length 369.81506
None None
song Too Shy artist Kajagoogoo length 223.55546
None None
song Potential artist The Jimmy Castor Bunch length 306.54649
None None
song The Great Satan (What Would Satan Do Mix) artist Ministry length 186.69669
None None
song The Screamer artist Bouncing Souls length 117.31546
None None
song Knife Going In (Album Version) artist Tegan And Sara length 129.67138
None None
song The Greatest View (Album Version) artist Silverchair length 242.96444
None None
song T.N.T. For The Brain (112 Bpm) (Radio Edit) artist Enigma length 242.78159
None None
song Mon MÃÂ©tro artist Anis length 278.12526
None None
song Fly Like An Eagle artist Seal length 254.06649
None None
song Restless artist 

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [23]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.